# Predicting Strokes from Admission Data

**In this exercise we are going to try to predict strokes from admission data using various machine learning models**

This exercise uses a [stroke dataset from Kaggle](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset).

Like other exercises, this is a classification problem - where we will try to determine if people have a stroke or not from some commonly used clinical metrics. We will also learn about the problem of _class imbalance_: a particularly common issue in healthcare - and learn about some approaches to over coming it. Furthermore, we will learn how to use data pipelines: an approach that helps us clean and and transform our data into a form which is more effective for machine learning algorithms to interpret and learn with.


In this exercise, we'll learn how to:
- **Download data** and **load it into our Jupyter Notebook**
- Import useful libraries like **pandas**, **sci-kit learn**, and **imbalanced-learn**
- **Clean our data**, and **modify it with sci-kit learn pipelines**
- Explore how we can manage **class imbalance**
- Quantify our model with **various metrics**

## Part 1: Downloading and Importing Data
To begin with, let's setup our notebook with the necessary packages as well as grab the data from Kaggle!

In [ ]:
# TODO: Establish venv/conda/package environment
# Commands with `%` run in the command line instead of within python so we don't have to do this within a seperate terminal!
# In this case, we are making sure our environment/colab instance has installed the latest versions of several commonly used data science packages
%pip install pandas numpy matplotlib seaborn imbalanced-learn

In [ ]:
# Setup matplotlib to display plots correctly within pandas
%matplotlib inline

# Import packages into our runtime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Download from Kaggle

Those who have downloaded and setup the kaggle CLI (Command Line Interface) can run the following command to download the dataset:

In [ ]:
# Download the stroke prediction dataset into the ./data folder
!kaggle datasets download -d fedesoriano/stroke-prediction-dataset --path ./data

## Part 2: Data Exploration
Now we have the data downloaded, we need to load this into a dataframe so we can explore it, and perform further analysis.

We can either do this by unzipping the dataset or just loading it directly with pandas:

In [ ]:
# No need to unzip the file - we can load the .csv file within directly into a DataFrame (commonly notated as `df`)
df = pd.read_csv("./data/stroke-prediction-dataset.zip")
df = df.rename({"Residence_type": "residence_type"}, axis=1)

It is good practice to explore what data we are actually dealing with and get a *feel* for it. We should check for missing data, what data is present, and relevant datatypes!

There are many ways of doing this with `pandas` ([API](https://pandas.pydata.org/docs/reference/frame.html#attributes-and-underlying-data)) - the most useful and commonly used methods being `head()`, `info()`, `describe()` initially:

In [ ]:
# Let's have a look at the first 10 entries of the dataset
df.head(10)

In [ ]:
df.info()

In [ ]:
# Describe is useful for looking at continuous data (i.e. int or float datatypes) - we can see it also has a look at data which is
# stored as booleans, as well as ID's (i.e. the id, hypertension, heart_disease, and stroke coluns)

# We can see
df.describe()

In [ ]:
# We can exclude the boolean or ID data by selecting certain rows for example

df[["age", "avg_glucose_level", "bmi"]].describe()

### Correlation and Graphing

We can explore the data further by looking at the correlation of datasets - in this case using a [Pearson correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient), which looks for **linear** correlations between variables (non-linear correlations will not be identified)

In [ ]:
corr = df.corr(numeric_only=True)
corr.style.background_gradient(cmap="coolwarm").format(precision=2)

#### Continuous Variables

In [ ]:
continuous_features = ["age", "avg_glucose_level", "bmi"]

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

for idx, feature in enumerate(continuous_features):
    sns.histplot(data=df, x=feature, ax=ax[idx], kde=True)
plt.show()

In [ ]:
df["stroke"].value_counts(normalize=True).plot(kind="pie", autopct="%.2f")
plt.xlabel("Stroke")
plt.ylabel("Percentage")

We can see that the vast majority of our dataset have not had a stroke - infact less than 5% have.

We will explore the implications of this later...

#### Discrete Variables

In [ ]:
discrete_features = ["ever_married", "work_type", "residence_type", "smoking_status"]

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(8, 8))

for idx, axis in enumerate(fig.axes):
    sns.histplot(df[discrete_features[idx]], ax=axis)
    axis.tick_params(labelrotation=30)


plt.show()


# plt.ylabel('Average Glucose Level (mg/dl)')
# plt.title('Comparing Average Glucose Level to Stroke Class')
# plt.show()

In [ ]:
df.groupby(["ever_married", "stroke"]).size().unstack().plot.bar()
plt.show()

## Part 3: Preparing our Data for Model Training
Now we have a feel for our dataset, we can begin building a useful model to try and predict outcomes

### Test-Train Split
We need to split our model into training sets (data which the machine learning algorithm uses to learn) and testing sets (used to validate how well our model is working).

### Data Pre-processing
In order to use our data - we will need to convert how our data is stored in order for machine learning models to interpret it. For example `Male` or `Female` can't be correctly interpreted.

Continuous data (i.e. BMI, Average Blood Glucose, and so on) will need to be scaled.


In [ ]:
df.isna().sum()

We can see we are missing 201 BMI values: at this point we have two options, to either drop the rows which are missing data, or to _impute_ their values. In this case, we will go for the latter!

There are more complex approaches to doing this - which can be done using the [`sklearn.imputer`](https://scikit-learn.org/stable/modules/impute.html) classes.

In [ ]:
# First calculate the average
mean_bmi = df["bmi"].mean()

# Now fill missing values with this mean
df["bmi"] = df["bmi"].fillna(mean_bmi)

# Check that our commands have worked!
df.isna().sum()

In [ ]:
# We can get rid of 'ID' column, this isn't needed!
df = df.drop(columns=["id"])

# We can then develop our `X` and `y` sets
X = df.drop(columns=["stroke"])
y = df["stroke"]

In [ ]:
# We can double check we have the right shape of data - we should have 5110 rows, with 10 features in the X set - and only a column of 1 or 0 in the y column
print(X.shape)
print(y.shape)

In [ ]:
# Perfect! Now to split into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

numerical_columns = ["age", "avg_glucose_level", "bmi"]
ordinal_columns = ["work_type", "smoking_status"]
binary_columns = ["gender", "ever_married", "residence_type"]

# These are columns we don't want to modify as they're already in a good form for ML training
pass_columns = ["hypertension", "heart_disease"]


numerical_transformer = make_pipeline(StandardScaler())
ordinal_transformer = make_pipeline(OneHotEncoder())
binary_transformer = make_pipeline(OrdinalEncoder())

pipeline = ColumnTransformer(
    [
        ("num", numerical_transformer, numerical_columns),
        ("ord", ordinal_transformer, ordinal_columns),
        ("bin", binary_transformer, binary_columns),
    ],
    remainder="passthrough",  # Include those we aren't changing in the pipeline
)

X_train_prep = pipeline.fit_transform(X_train)

In [ ]:
prep_df = pd.DataFrame(X_train_prep)
prep_df.columns = pipeline.get_feature_names_out()
prep_df.head()

In [ ]:
# Now we need to scale the test data which we haven't yet touched
# NB: We only TRANSFORM the test date - we don't want to fit our transformers to this data
X_test_prep = pipeline.transform(X_test)

## Part 4: Making our First Model

Now we have imported, cleaned, and transformed our data - it is finally in a form that we can make models to help us predict future strokes.

To begin with, we will use a Logistic Regression to make these predictions.

In [ ]:
from sklearn.linear_model import LogisticRegression

# TODO: Build first models
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train_prep, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, f1_score

# we can now predict some strokes in our test dataset - which our model has not seen!
y_pred = logistic_regression.predict(X_test_prep)
print(accuracy_score(y_test, y_pred))

#### 🤔 Something Fishy Afoot

Hmmm... our model has a pretty a pretty good "accuracy" - however if we look deeper something odd is going on.

Let's have a look at what are model is guessing

In [ ]:
y_pred.sum()  # Add's up how many 1's there are (i.e. how many people we have predicted have had a stroke)

Essentially our model just guesses `0` / no stroke every time and get's a pretty good accuracy.

For imbalanced class problems (a common and important issue in medicine!) we need to dive deeper other metrics are more useful to understand how well our model truly works. You may remember from previous exercises the F1 Score ~ which provides a metric of how well a model is at making true positives/negatives and false positives/negatives. Unfortunately, there will always be a trade off between these - and one we need to keep in mind when designing machine learning models in healthcare.

The F1 score is a *harmonic mean* of the Precision and Recall:

$$\text{F1 Score} = 2\times\frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} = \frac{2TP}{2TP + FP + FN}$$

We have to make decisions when we tune our model if we want more *false positives* or more *false negatives* - with real world implications. If we were deploying our model, we would have to decide if we would rather have fewer false positives - and the run risk of missing potential strokes - or predict more people were having strokes when they aren't, and run the risk of over investigation and unrequired treamtment/intervention.

If you would like to learn more about this it is well worth looking at the [Machine Learning University's explainer on Precision and Recall](https://mlu-explain.github.io/precision-recall/) which nicely demonstrates how the precision-recall tradeoff works visually.

In [ ]:
print(f"F1 Score: {f1_score(y_test, y_pred)}")

The F1 Score is 0 in this case, as the model doesn't predict any cases of stroke - and so the numerator is 0 (no True Positives!).

Let's check more complicated models also suffer from the same issue - to check that this isn't purely as a result of [Logistic Regression](https://mlu-explain.github.io/logistic-regression/) being unable to capture complicated relationships.

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

svm = SVC()
forest = RandomForestClassifier()
gbm = GradientBoostingClassifier()

for model in (svm, forest, gbm):
    model.fit(X_train_prep, y_train)
    y_pred = model.predict(X_test_prep)

    print(f"F1 Score ({type(model).__name__}): {f1_score(y_test, y_pred)}")

## Part 5: Tackling Unbalanced Datasets
An unbalanced dataset is where a *class* or outcome is much less common than another: in this case the vast majority of people don't have strokes. This can lead machine learning models to tend to predict a common class (`no stroke`) over a minority/rare class (`stroke`) as it has receives many more examples of one class over another, and is unable to learn what differentiates the two.

There are various ways of approaching unbalanced datasets like these, which we will explore now:

### Get More Data
In an ideal world, we could simply collect more data to represent the *minority* class, however this is not always easy or feasible. We would have to wait many months or years to collect lots of data for strokes for example!

### Weighting/Model Penalization
We can 'penalize' our model to put a greater weighting on a minority class. In essence, this forces the model to pay more attention to the minority class

### Resampling
Essentially we are manipulating our dataset so there is a ratio of classes closer to 50:50. There are two approaches within this:
 1. **Undersampling** - where we take a sample (either randomly, or through more intelligent means) of the **majority** class, so that the ratio is limited, or
 2. **Oversampling** - where we expand the number of **minority** class: we can do this by duplicating instances of the minority class, or we can synthesise examples of this class.

This is nicely visualised below <sup>([ref](https://blog.strands.com/unbalanced-datasets))</sup>

<img src="https://blog.strands.com/hs-fs/hubfs/Screenshot%202019-07-18%20at%2014.15.15.png?width=1200&name=Screenshot%202019-07-18%20at%2014.15.15.png" width=700>




### Getting More Data

Sadly, we can't get more data in this case - but what we can do is make sure the *train* and *test* set have equal numbers of each class. Fortunately `sklearn` makes this easy to do when we split our model, so we don't have to worry about one size being bigger than the other!

In [ ]:
# In this case - we are forcing the percentages of strokes to be the same in the train and test class
# This is done be setting the stratify command to our outcome column of strokes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# We now need to pass our data through our pipeline again!
X_train_prep = pipeline.fit_transform(X_train)
X_test_prep = pipeline.transform(X_test)

Sadly, this is unlikely to make much difference - as by pure chance there would likely be a similar number of strokes in the test and train set anyway.

### Model Penalisation

We can teach our model to pay more attention to each class by putting a 'weight' on each class. Fortunately this is easy to do in sklearn with the `class_weight` attribute!

In [ ]:
# === Class Weighting ===
# We will begin with adjusting class weights as this is easy to do in the first instance
logistic_regression = LogisticRegression(class_weight="balanced")

# Fit our model
logistic_regression.fit(X_train_prep, y_train)

# Make some predictions
y_pred = logistic_regression.predict(X_test_prep)

# See how well it performs
print(f"Accuracy Score (Logistic Regression): {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 Score (Logistic Regression): {f1_score(y_test, y_pred):.4f}")

In [ ]:
# Lets try more complicated models
svm = SVC(class_weight="balanced")
forest = RandomForestClassifier(class_weight="balanced")

for model in (svm, forest):
    model.fit(X_train_prep, y_train)
    y_pred = model.predict(X_test_prep)

    print(f"F1 Score ({type(model).__name__}): {f1_score(y_test, y_pred)}")

We can see that we finally have an F1 score greater than 0m, and our Logistic regression model is performing the best at the moment. 

### Under and Over Sampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

under = RandomUnderSampler(sampling_strategy=0.1)
over = RandomOverSampler(sampling_strategy=0.5)
model = LogisticRegression()

X_under, y_under = under.fit_resample(X_train_prep, y_train)
X_combined, y_combined = over.fit_resample(X_under, y_under)

In [ ]:
logreg = LogisticRegression()
svm = SVC()
forest = RandomForestClassifier()
knn = KNeighborsClassifier()
gbm = GradientBoostingClassifier()

for model in (logreg, svm, forest, knn, gbm):
    model.fit(X_combined, y_combined)
    y_pred = model.predict(X_test_prep)

    print(f"F1 Score ({type(model).__name__}): {f1_score(y_test, y_pred)}")

## Part 6: Under/Over-Fitting

In [ ]:
# TODO: Explore under/overfitting.